In [ ]:
#pymysql because The error indicates that the MySQL server uses the caching_sha2_password authentication plugin, which is not supported by the version of the mysql-connector Python library you're using.

#This happens because newer versions of MySQL default to caching_sha2_password, while some connectors only support the older mysql_native_password.
#pip install pymysql


In [ ]:
import pandas as pd
import mysql.connector
import os


In [ ]:
csv_files=[('customers.csv','customers'),
           ('payments.csv','payments'),
          ('orders.csv','orders'),
          ('order_items.csv','order_items'),
          ('geolocation.csv','geolocation'),
          ('sellers.csv','sellers'),
          ('products.csv','products')]

In [ ]:
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Lhrsd1234#',
    database='ecommerce'
)
cursor = conn.cursor()

In [ ]:
#Folder containing csv files
folder_path='C:/Users/User/Downloads/archive (5)'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()